In [80]:
from random import randint
from sympy import isprime, mod_inverse

In [81]:
def Gen(q, g):
    if not isprime(q):
        raise ValueError("q debe ser un número primo.")
    alpha = randint(2, q - 1)
    u = pow(g, alpha, q)
    pk = u
    sk = alpha
    return pk, sk

In [82]:
q = 178286657344291
g = 10

pk, sk = Gen(q, g)

In [83]:
def Enc(pk, m, q, g):
    if not isinstance(m, str):
        raise TypeError("El mensaje debe ser una cadena de texto.")

    # Determinar el tamaño de cada submensaje
    k = len(m) // 4
    if len(m) % 4 != 0:  # Ajustar si no es un múltiplo exacto de 4
        k += 1

    # Dividir el mensaje en submensajes
    submensajes = [m[i * 4: (i + 1) * 4] for i in range(k)]

    resultados = []
    for submensaje in submensajes:
        # Convertir submensaje a un número entero
        submensaje_int = int.from_bytes(submensaje.encode(), 'big')
        if not (1 < submensaje_int < q):
            raise ValueError(f"El valor entero del submensaje '{submensaje}' ({submensaje_int}) debe estar en el rango de 1 a q-1.")
        
        beta = randint(2, q - 1)
        v = pow(g, beta, q)
        c = (submensaje_int * pow(pk, beta, q)) % q
        resultados.append((beta, v, c))

    return resultados

In [84]:
# Mensaje a cifrar
m = '-Life before death.-'

In [85]:
ci = []
# Cifrado del mensaje
try:
    resultados = Enc(pk, m, q, g)
    for i, (beta, v, c) in enumerate(resultados):
        print(f"(beta_{i}, v_{i}, c_{i}) = ({beta}, {v}, {c})")
        ci.append((v, c))
except ValueError as e:
    print(f"Error: {e}")

(beta_0, v_0, c_0) = (173173960281961, 107339606284172, 73201671705625)
(beta_1, v_1, c_1) = (149454926476970, 28397948415576, 99599063935425)
(beta_2, v_2, c_2) = (122807412151923, 108078851462668, 90416930996292)
(beta_3, v_3, c_3) = (102058138942299, 70375825011039, 34081462167948)
(beta_4, v_4, c_4) = (71882225541629, 19239058113309, 30912818269314)


In [86]:
def Dec(sk, ciphers, q):
    message_parts = []

    for v, c in ciphers:
        w = pow(mod_inverse(v, q), sk, q)
        m_int = (c * w) % q
        try:
            # Calculamos el número de bytes a partir del valor máximo esperado en el cifrado
            num_bytes = max(1, (m_int.bit_length() + 7) // 8)
            m_bytes = m_int.to_bytes(num_bytes, byteorder='big')
            m_part = m_bytes.decode('utf-8', errors='ignore')  # Ignorar caracteres no decodificables
            message_parts.append(m_part)
        except (ValueError, UnicodeDecodeError, OverflowError) as e:
            message_parts.append(f"<cannot decode {m_int}: {str(e)}>")

    return ''.join(message_parts)

In [88]:
decrypted_message = Dec(sk, ci, q)

decrypted_message

'-Life before death.-'